## Exercise 01. Subquery ##

## Imports ##

In [22]:
import pandas as pd
import sqlite3

## Task ##

1. Создайте соединение с базой данных, используя библиотеку "sqlite3".

2. Получите схему таблицы "checker".

3. Получите только первые десять строк таблицы, чтобы понять её структуру.

4. Посчитайте, сколько строк удовлетворяют следующим условиям, используя один запрос с любым количеством подзапросов:
   - Подсчитайте строки из таблицы "pageviews", но только те, что относятся к таблице "checker" с:
     - "status = 'ready'", чтобы не анализировать логи, находящиеся в процессе проверки статуса.
     - "numTrials = 1", то есть только первые попытки, поскольку они показывают, когда студент начал работать над лабораторной работой.
     - Названия лабораторных "labnames" должны входить в список: "laba04", "laba04s", "laba05", "laba06", "laba06s" и "project1". Только эти назывались во время эксперимента.
   - Сохраните результаты подсчёта в датафрейм с колонкой "cnt".
   
5. Закройте соединение.

In [23]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
schema = pd.read_sql('PRAGMA table_info(checker);', conn)
print(schema)

   cid       name       type  notnull dflt_value  pk
0    0      index    INTEGER        0       None   0
1    1     status       TEXT        0       None   0
2    2    success    INTEGER        0       None   0
3    3  timestamp  TIMESTAMP        0       None   0
4    4  numTrials    INTEGER        0       None   0
5    5    labname       TEXT        0       None   0
6    6        uid       TEXT        0       None   0


In [24]:
first_ten = pd.read_sql('SELECT * FROM checker LIMIT 10;', conn)
first_ten.head(10)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [25]:
query_result = pd.read_sql('SELECT pageviews.* FROM (SELECT * FROM checker WHERE status="ready" AND numTrials=1 AND labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1")) AS t1 LEFT JOIN pageviews ON t1."index" = pageviews."index";', conn)
print(query_result.head())
cnt = pd.DataFrame(query_result.count(), columns=['cnt'])
cnt

   index      uid                    datetime
0    7.0  admin_3  2020-04-17 12:36:21.401412
1   21.0  user_10  2020-04-18 12:19:50.182714
2   27.0  user_14  2020-04-18 18:20:37.860671
3   33.0  user_14  2020-04-18 21:14:43.989325
4   35.0  user_14  2020-04-18 21:36:38.965506


,cnt
index,74
uid,74
datetime,74


In [26]:
conn.close()